In [53]:
# for accessing file system
import os

# for numerical work
import numpy as np

# for loading netcdf files, for metadata
import xarray as xr

# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy,
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

# Used for processing netCDF time data
import requests

# Used for indexing via metadata
import pandas as pd

# local modules
from util import *

os.environ["VISUS_CACHE"] = "./visus_cache_can_be_erased"
os.environ["CURL_CA_BUNDLE"] = ""

# whether to download tiny netcdf or not
download = False

local_netcdf = "firesmoke_metadata.nc"

if download:
    # path to tiny NetCDF
    url = "https://github.com/sci-visus/NSDF-WIRED/raw/main/data/firesmoke_metadata.nc"

    # Download the file using requests
    response = requests.get(url)
    with open(local_netcdf, "wb") as f:
        f.write(response.content)

# open tiny netcdf with xarray and OpenVisus backend
ds = xr.open_dataset(local_netcdf, engine=OpenVisusBackendEntrypoint)

##### Process all metadata of file #####
### Lat/Lon ###
# get metadata to compute lon and lat
xorig = ds.XORIG
yorig = ds.YORIG
xcell = ds.XCELL
ycell = ds.YCELL
ncols = ds.NCOLS
nrows = ds.NROWS

longitude = np.linspace(xorig, xorig + xcell * (ncols - 1), ncols)
latitude = np.linspace(yorig, yorig + ycell * (nrows - 1), nrows)

# Create coordinates for lat and lon (credit: Aashish Panta)
ds.coords["lat"] = ("ROW", latitude)
ds.coords["lon"] = ("COL", longitude)

# Replace col and row dimensions with newly calculated lon and lat arrays (credit: Aashish Panta)
ds = ds.swap_dims({"COL": "lon", "ROW": "lat"})

### Timestamps ###
# get all tflags
tflag_values = ds["TFLAG"].values

# to store pandas timestamps
timestamps = []

# convert all tflags to pandas timestamps, store in timestamps list
for tflag in tflag_values:
    timestamps.append(pd.Timestamp(parse_tflag(tflag[0])))

# set coordinates to each timestep with these pandas timestamps
ds.coords["time"] = ("time", timestamps)


##### Functions for bokeh to access data #####
def get_latslons():
    """
    Return a numpy array of all lats and lons used in dataset in mercator coordinates
    """
    # get list of latitudes and longitudes
    data_stacked_index = ds["PM25"][0].stack(lat_lon=["lat", "lon"])

    # numpy array to return
    mercator_latlons = np.zeros((np.shape(data_stacked_index)[1], 2))
    for i, tup in enumerate(latlon_to_mercator_iter(data_stacked_index.lat_lon.values)):
        mercator_latlons[i][0] = tup[0]
        mercator_latlons[i][1] = tup[1]

    return mercator_latlons


def get_pm25(date, res):
    """
    Return a numpy array of pm2.5 values for all hours of given date at specified resolution
    :param string date: Date to query data for, in format "YYYY-MM-DD"
    :param int res: IDX resolution to use
    """
    # get time range
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[-2:])
    start_time = get_timestamp(year, month, day, 0)
    end_time = get_timestamp(year, month, day, 23)
    # get slice
    data_array_at_time = ds["PM25"].loc[start_time:end_time, :, :, res]
    return data_array_at_time.values


ov.LoadDataset(http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1)
PM25
Adding field  PM25 shape  [27357, 381, 1081, 21] dtype  float32 labels  ['time', 'ROW', 'COL', 'resolution'] Max Resolution  20


In [54]:
arr = get_pm25("2021-03-04", 0)

Using Max Resolution:  20
Time: 0, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 1, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 2, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 3, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 4, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 5, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 6, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 7, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 8, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 9, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 10, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 11, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 12, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25
Time: 13, max_resolution: 20, logic_box=(0, 1081, 0, 381), fi

In [55]:
np.shape(arr)

(24, 381, 1081)